# Bushe Customer behaviour analysis - Classifiers


<h4>Цель этого исследования - понять поведение клиентов, которые останутся верными  Bushe.
Решение задачи будет осуществляться путем построения классификатора.
В этом Kernel мы будем использовать различные модели классификации,<br> чтобы увидеть, насколько они точны при определении того, <br>будет ли посетитель с определенным набором параметров посещать Bushe снова.</center>
<br><h3>Алгоритм выполнения процедур построения классификатора <br>(*кликнув на выделенный элемент, можно перейти сразу к нужному блоку  в структуре кернел)</h3>

* [Обзор датафрейма](#aaa)<br>
* [Преобразование категориальных признаков в числовые значения](#qqq)
<br>В начале будет проведено преобразование категориальных признаков в числовые значения.Что приведет к увеличению числа независимых предикторов - их станет 63. Далее будет произведена стандаритизация и нормирование.

Чтобы выбрать предикторы, которые сильнее объясняли бы поведение зависсимой переменной Y (в нашем случае это вреятность возврата гостя в BUshe -'Comeback') мы предлагаем три варианта:

1. [Выполнение процедуры Оценки Коэффициента Корреляции Пирсона и нахождение уровня значимости P-value.](#eee)

2. [GridSearchCV –  инструмент для автоматического подбирания параметров для моделей машинного обучениялучшие оценки с оптимальной комбинацией гиперпараметров.](#yyy)Определяет какие параметры лучше всего работают со случайным поиском, и формируем на их основе сетку, чтобы увидеть, сможем ли мы найти лучшую комбинацию.

3. PCA - это метод преобразования исходного набора данных, представленного векторными выборками, в новый набор векторных образцов (или главных компонент) с производными размерами.Позволяет легко уменьшить размерность структуры данных в нескольких главных компонентах без эффекта на данные.(* PCA в этом исследовании пока не рассмотрено)



**[Выявление наиболее значимых Предикторов, проявляющих  сильное влияние на показатель "Comeback" - возвращения гостя в Bushe](#prj)** 
    
    
[+**Классификация**](#rrr)

В этом разделе мы обучим четыре типа классификаторов и решим, какой классификатор будет более эффективным при обнаружении поведения посетителей. Прежде всего нам нужно разделить наши данные на наборы для обучения и тестирования

* K Nearest Neighbor(KNN)
* Decision Tree
* Support Vector Machine
* Logistic Regression
<br><br>
**[Кривые обучения -learning Curves](#iii) Позволяют определить наличие переобучения модели, и наглядно демонстрируют точность. [Было выявлено, Классификатор Логистической регрессии, а также SVM показывают лучший результат как в обучающих наборах, так и в Cross-validation проверке](#ooo)

*[ROC Кривая ошибок](#hhh)

*[Построение Классификатора на основе Логистической Регресии](#kkk)

*[Confusion Matrix](#cmm)
* [Проверка работоспособности классификатора](#nnn)

* [Оценка Точности и Меры сходства**](#msx)


     
     

In [ ]:
# Imported Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections

# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_raw = pd.read_csv('../input/busheclassique/Busheclassic.csv')
df_raw
dff = pd.read_csv('../input/starbucks-customer-retention-malaysia-survey/Starbucks satisfactory survey.csv')
df_raw['membership']=dff['9. Do you have Starbucks membership card?']
df_raw['Comeback']=dff['20. Will you continue buying at Starbucks?']

In [ ]:
df_new = df_raw.rename(columns={"1. Your Gender": 'Gender',
                                "2. Your Age": "ageGroup",
                                "3. Are you currently....?":"status",
                                "4. What is your annual income?":"annualIncome",
                                "5. How often do you visit Bushe?":"visitFrequency",
                                "6. How do you usually enjoy Bushe?":"Method",
                                "7. How much time do you normally  spend during your visit?":"timeCost",
                                "8. The nearest Bushe's outlet to you is...?":"location",
                                "10. What do you most frequently purchase at Bushe?":"mostPurchase",
                                "11. On average, how much would you spend at Bushe per visit?":"moneySpend",
                                "12. How would you rate the quality of Bushe compared to other brands (Coffee Bean, Old Town White Coffee..) to be:":"ComparingRate",
                                "13. How would you rate the price range at Bushe?":"PriceRate",
                                "14. How important are sales and promotions in your purchase decision?":"s&pCareness",
                                "15. How would you rate the ambiance at Bushe? (lighting, music, etc...)":"AmbientRate",
                                "16. You rate the WiFi quality at Bushe as..":"WifiRate",
                                "17. How would you rate the service at Bushe? (Promptness, friendliness, etc..)":"ServiceRate",
                                '19. How do you come to hear of promotions at Bushe? Check all that apply.':'Promo Source',
                                "18. How likely you will choose Bushe for doing business meetings or hangout with friends?":"MhRating",
                                "20. bakery":"bakery",
                                'membership':'membership',
                                'Comeback': 'Comeback'
                        
                                 }, inplace=False)
df_new.drop('21. Will you continue buying at Bushe?',
axis='columns', inplace=True)
df_new.drop('9. Do you have Bushe membership card?',
axis='columns', inplace=True)

<a id="aaa"></a>
# Так выглядит наш анализируемый датасет
122 позиций по 22 столбцам. Зависсимой переменной будет приходиться последний столбец ['Comeback']

In [ ]:
df=df_new
df

<a id="qqq"></a>
# Преобразование категориальных признаков в числовые значения
В нашем датасете полно колонок с категорийными признаками где присутствует и текст и числовые комбинации.
Преобразование категориальных признаков в числовые значения. Нам необходимо все признаки перевести в числовые значения.
<br><br>Взглянем на распределение Лояльных признаков по половому признаку

In [ ]:
colors = ["#0101DF", "#DF0101"]

sns.countplot('Comeback', data=df, palette=colors)
plt.title('Class Distributions \n (0:Loyal|| 1: Lost)', fontsize=14)

df.groupby(['Gender'])['Comeback'].value_counts(normalize=True)

76% опрошенных мужчин пожелали бы вернуться, против 77% женщин. Что практически одинаково <br>
Преобразуем мужской пол в 0, а женский в 1:

In [ ]:
df['Gender'].replace(to_replace=['Male','Female'], value=[0,1],inplace=True)
df['membership'].replace(to_replace=['Yes','No'], value=[1,0],inplace=True)
df['Comeback'].replace(to_replace=['Yes','No'], value=[1,0],inplace=True)

In [ ]:

df['Method'].replace(to_replace=['never  ','Never buy'], value=['Never','Never'],inplace=True)
df['Method'].replace(to_replace=['I dont like coffee','Never buy'], value=['Never','Never'],inplace=True)
df['Method'].replace(to_replace=['I dont like coffee','never'], value=['Never','Never'],inplace=True)
df['Method'].replace(to_replace=['Never ','never'], value=['Never','Never'],inplace=True)
df['Method'].replace(to_replace=['Never '], value=['Never'],inplace=True)
df['Method'].value_counts()

# Scaling and Distributing
Стандартизация данных дает нулевое среднее значение и единичную дисперсию


In [ ]:
Feature = df[['membership','PriceRate','s&pCareness','AmbientRate','WifiRate','ServiceRate','MhRating']]
Feature = pd.concat([Feature,pd.get_dummies(df['mostPurchase'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['ageGroup'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['status'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['annualIncome'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['visitFrequency'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['timeCost'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['location'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['Method'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['bakery'])], axis=1)
Feature.head(5)

In [ ]:
#Fh=Feature
#Fh['Comeback']=df['Comeback']
#Fh.to_csv('train.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
Ftt=Feature
#Ftt=Ftt.loc[:, Ftt.columns != 'Comeback']
scaler = StandardScaler()
df_sc= pd.DataFrame(scaler.fit_transform(Ftt.values), columns=Ftt.columns)
df_sc['Comeback']=df['Comeback']

# FAQ
Столбец **Comeback** представляет  собой ответы респондентов на вопрос :'20. Will you continue buying at Bushe?', который имеет бинарный параметр Yes или NO. **Comeback** В нашем классификаторе будет играть роль зависсимой переменной Y, поведение которой и будет моделироваться 

In [ ]:
df_sc = df_sc.sample(frac=1)

# amount of fraud classes 492 rows.
comeback_df = df_sc.loc[df_sc['Comeback'] == 1]
non_comeback_df = df_sc.loc[df_sc['Comeback'] == 0][:122]

normal_distributed_df = pd.concat([comeback_df, non_comeback_df])

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=22)

new_df

<a id="eee"></a>
# **Строим Матрицу корреляций**
****

In [ ]:
df_corr=new_df.corr()
df1 = pd.DataFrame(df_corr.Comeback)
df1.sort_values(by=['Comeback'], inplace=True)
print(df1.to_markdown())

In [ ]:
 
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,20))
# Entire DataFrame
corr = df.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)
ax1.set_title('Imbalanced Correlation Matrix', fontsize=14)


sub_sample_corr = new_df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2)
ax2.set_title('SubSample Correlation Matrix \n (use for reference)', fontsize=14)
plt.show()

<a id="prj"></a>
<h2>Предикторы, которые показали положительную значимость с оценивающим
    показателем "Comeback" по Пирсону и P-value</h2> 
                                                 
         

Имеют статистическую значимость по показателю теста  P-value <0,005
* | Параметр                                      |  Pearson    |   P-value
* | Monthly                                       |  0.236433   |  0.00874456029197296
* | AmbientRate                                   |  0.318145   |  0.0003554891498241922
* | membership                                    |  0.341955   |  0.00011572126682106971
* | MhRating                                      |  0.369625   |  2.789627898236357e^-5
* | PriceRate                                     |  0.452593   |  1.6600899812402898e^-7

где


* MhRating - это  Насколько вероятно, что вы выберете Буше для деловых встреч или встреч с друзьями?
* AmbientRate -Как бы вы оценили атмосферу в Буше? (освещение, музыка и т. д.)
* PriceRate  - Как бы вы оценили диапазон цен в Буше?
* membership - наличие накопительных карт
* Monthly  - посещает раз в месяц

<a id="rrr"></a>
# Classification
В этом разделе мы обучим четыре типа классификаторов и решим, какой классификатор будет более эффективным при обнаружении поведения посетителей. Прежде всего нам нужно разделить наши данные на наборы для обучения и тестирования 

* K Nearest Neighbor(KNN)
* Decision Tree
* Support Vector Machine
* Logistic Regression



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing

In [ ]:
X = new_df.drop('Comeback', axis=1)
y = new_df['Comeback']

from sklearn.model_selection import train_test_split


#Установка фиксированного значения random_state используется для инициализации внутреннего генератора 
#случайных чисел гарантирует, 
#что при каждом запуске кода будет генерироваться одна и та же последовательность случайных чисел
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)


In [ ]:
# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values


# Объявим  классификаторы - Алгортимы которые будем использовать

In [ ]:

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [ ]:
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

<h4>Педварительная оценка подогнанных моделей, распределилась таким образом, что  KNeighborsClassifier Has a training score около 80 % accuracy score. Не будем на этом останавливаться. Подберем лучшие параметры для обучения и оценим модели на наличие переобучения

<a id="yyy"></a>
# GridSearchCV 
это очень мощный инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров.
**используется для определения параметров, которые дают лучший прогноз для классификаторов**.
Полученные параметры являются лучшими для нашей модели.
**grid.best_params_ подберем по всем классификаторам, что в итоге, должно увеличить качество и точность классификатора.


In [ ]:
from sklearn.model_selection import GridSearchCV


# Logistic Regression 
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}


grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train, y_train)
# We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_

knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(X_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_

# Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
grid_svc = GridSearchCV(SVC(), svc_params)
grid_svc.fit(X_train, y_train)

# SVC best estimator
svc = grid_svc.best_estimator_

# DecisionTree Classifier
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))}
grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_params)
grid_tree.fit(X_train, y_train)

# tree best estimator
tree_clf = grid_tree.best_estimator_
# Overfitting Case

log_reg_score = cross_val_score(log_reg, X_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')


knears_score = cross_val_score(knears_neighbors, X_train, y_train, cv=5)
print('Knears Neighbors Cross Validation Score', round(knears_score.mean() * 100, 2).astype(str) + '%')

svc_score = cross_val_score(svc, X_train, y_train, cv=5)
print('Support Vector Classifier Cross Validation Score', round(svc_score.mean() * 100, 2).astype(str) + '%')

tree_score = cross_val_score(tree_clf, X_train, y_train, cv=5)
print('DecisionTree Classifier Cross Validation Score', round(tree_score.mean() * 100, 2).astype(str) + '%')

<h4>**Покзатели точности для Логистической регресси и SVM улучшились**

<a id="iii"></a>
# Кривые обучения:

Построение кривых обучения, нужно для наглядного обхяснения, того, а какой алгоритм справился с классификацией точнее, и без переобучений
Чем шире разрыв между оценкой обученной моедли и оценкой  проверки cross validation, тем больше вероятность того, что ваша модель переобучена (высокая дисперсия).

Если результат низкий как в обучающем наборе, так и в наборе cross validation, это свидетельствует о том, что наша модель не соответствует требованиям (высокая систематическая ошибка).

In [ ]:
# Let's Plot LogisticRegression Learning Curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator1, estimator2, estimator3, estimator4, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(20,14), sharey=True)
    if ylim is not None:
        plt.ylim(*ylim)
    # First Estimator
    train_sizes, train_scores, test_scores = learning_curve(
        estimator1, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax1.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="#ff9124")
    ax1.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
    ax1.plot(train_sizes, train_scores_mean, 'o-', color="#ff9124",
             label="Training score")
    ax1.plot(train_sizes, test_scores_mean, 'o-', color="#2492ff",
             label="Cross-validation score")
    ax1.set_title("Logistic Regression Learning Curve", fontsize=14)
    ax1.set_xlabel('Training size (m)')
    ax1.set_ylabel('Score')
    ax1.grid(True)
    ax1.legend(loc="best")
    
    # Second Estimator 
    train_sizes, train_scores, test_scores = learning_curve(
        estimator2, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax2.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="#ff9124")
    ax2.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
    ax2.plot(train_sizes, train_scores_mean, 'o-', color="#ff9124",
             label="Training score")
    ax2.plot(train_sizes, test_scores_mean, 'o-', color="#2492ff",
             label="Cross-validation score")
    ax2.set_title("Knears Neighbors Learning Curve", fontsize=14)
    ax2.set_xlabel('Training size (m)')
    ax2.set_ylabel('Score')
    ax2.grid(True)
    ax2.legend(loc="best")
    
    # Third Estimator
    train_sizes, train_scores, test_scores = learning_curve(
        estimator3, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax3.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="#ff9124")
    ax3.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
    ax3.plot(train_sizes, train_scores_mean, 'o-', color="#ff9124",
             label="Training score")
    ax3.plot(train_sizes, test_scores_mean, 'o-', color="#2492ff",
             label="Cross-validation score")
    ax3.set_title("Support Vector Classifier \n Learning Curve", fontsize=14)
    ax3.set_xlabel('Training size (m)')
    ax3.set_ylabel('Score')
    ax3.grid(True)
    ax3.legend(loc="best")
    
    # Fourth Estimator
    train_sizes, train_scores, test_scores = learning_curve(
        estimator4, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax4.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="#ff9124")
    ax4.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
    ax4.plot(train_sizes, train_scores_mean, 'o-', color="#ff9124",
             label="Training score")
    ax4.plot(train_sizes, test_scores_mean, 'o-', color="#2492ff",
             label="Cross-validation score")
    ax4.set_title("Decision Tree Classifier \n Learning Curve", fontsize=14)
    ax4.set_xlabel('Training size (m)')
    ax4.set_ylabel('Score')
    ax4.grid(True)
    ax4.legend(loc="best")
    return plt


<a id="ooo"></a>
# Классификатор Логистической регрессии, а также SVM показывают лучший результат как в обучающих наборах, так и в Cross-validation проверке.

In [ ]:
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
plot_learning_curve(log_reg, knears_neighbors, svc, tree_clf, X_train, y_train, (0.67, 1.01), cv=cv, n_jobs=4)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
# Create a DataFrame with all the scores and the classifiers names.

log_reg_pred = cross_val_predict(log_reg, X_train, y_train, cv=5,
                             method="decision_function")

knears_pred = cross_val_predict(knears_neighbors, X_train, y_train, cv=5)

svc_pred = cross_val_predict(svc, X_train, y_train, cv=5,
                             method="decision_function")

tree_pred = cross_val_predict(tree_clf, X_train, y_train, cv=5)

from sklearn.metrics import roc_auc_score

print('Logistic Regression: ', roc_auc_score(y_train, log_reg_pred))
print('KNears Neighbors: ', roc_auc_score(y_train, knears_pred))
print('Support Vector Classifier: ', roc_auc_score(y_train, svc_pred))
print('Decision Tree Classifier: ', roc_auc_score(y_train, tree_pred))


<a id="hhh"></a>
#  ROC Кривая ошибок 
графичекая характеристика качества бинарного классификатора. наглядно представляет, каким будет качество классификации при различных варьированиях порога

In [ ]:
log_fpr, log_tpr, log_thresold = roc_curve(y_train, log_reg_pred)
knear_fpr, knear_tpr, knear_threshold = roc_curve(y_train, knears_pred)
svc_fpr, svc_tpr, svc_threshold = roc_curve(y_train, svc_pred)
tree_fpr, tree_tpr, tree_threshold = roc_curve(y_train, tree_pred)


def graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr):
    plt.figure(figsize=(16,8))
    plt.title('ROC Curve \n Top 4 Classifiers', fontsize=18)
    plt.plot(log_fpr, log_tpr, label='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_train, log_reg_pred)))
    plt.plot(knear_fpr, knear_tpr, label='KNears Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_train, knears_pred)))
    plt.plot(svc_fpr, svc_tpr, label='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_train, svc_pred)))
    plt.plot(tree_fpr, tree_tpr, label='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_train, tree_pred)))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.01, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.annotate('Minimum ROC Score of 50% \n (This is the minimum score to get)', xy=(0.5, 0.5), xytext=(0.6, 0.3),
                arrowprops=dict(facecolor='#6E726D', shrink=0.05),
                )
    plt.legend()
    
graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr)
plt.show()

<h3>Площадь под ROC-кривой AUC (Area Under Curve) является агрегированной характеристикой качества классификации, не зависящей от соотношения цен ошибок. Чем больше значение AUC, тем «лучше» модель классификации.</h3>



<a id="kkk"></a>
# Logistic Regression¶

<h2>В ходе проверки моделей, было выявлено о превосходстве линейного байесовского классификатора (логистической регрессии)</h2>
Дальнейшую процедуру классификации, и последующие верификации модели будут проводиться исключительно на этом классификаторе  LR

In [ ]:
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


<h3>Выбираем лучшие параметры из GridSearchCV grid_log_reg.best_estimator_ который мы получили ранее


In [ ]:

grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train, y_train)
# We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_
LR=log_reg
LR

In [ ]:
yhat = LR.predict(X_test)
yhat
yhat_prob = LR.predict_proba(X_test)
yhat_prob

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
#print(confusion_matrix(y_test, yhat))
print(confusion_matrix(y_test, yhat, labels=[1,0]))

<a id="cmm"></a>
# Confusion Matrix

In [ ]:
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Я приду в Буше еще','Это вряд ли'],normalize= False,  title='Confusion matrix')

In [ ]:
print (classification_report(y_test, yhat))

<a id="nnn"></a>
# ПРОВЕРКА РАБОТОСПОСОБНОСТИ КЛАССИФИКАТОРА

Загрузим новую таблицу из 9 строк и попробуем оценить, как склассифицирует респондентов обученная модель 

In [ ]:
df_checker = pd.read_csv('../input/lr-check/cheker_LR.csv')

df_checkerLR=df_checker.loc[:, df_checker.columns != 'Comeback']


In [ ]:
df_checkerLR

In [ ]:
#для проверки
t_y = df_checker['Comeback'].values
t_y[0:9]

**МОДЕЛИРОВАНИЕ**

In [ ]:

t_yhatLR = LR.predict(df_checkerLR)
t_yhatLR_prob = LR.predict_proba(df_checkerLR)


**Результат классифицирования**

In [ ]:
print(t_yhatLR) 

# **Результат в виде оценки Вероятностей.**

In [ ]:
LR_prob=pd.DataFrame(t_yhatLR_prob)
LR_prob

<a id="msx"></a>
# **Оценка Точности и Меры сходства**
* Коэффициент Жаккара - отношение количества элементов пересечения множеств количеству элементов их объединения
* Метрика F1 раскрывает одновременно и точность и полноту 

In [ ]:
print("ОЦЕНКА КАЧЕСТВА ПОСТРОЕННОГО КЛАССИФИКАТОРА" )
LGR_Jaccard = jaccard_score(t_y, t_yhatLR)
print("Logistic Regression - Jaccard accuracy = " , LGR_Jaccard)

# f1_score
LGR_f1_score = f1_score(t_y, t_yhatLR , average='weighted') 
print("Logistic Regression - F1 score accuracy = " , LGR_f1_score)


<a id="finito"></a>
# Спасибо за внимание !